In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu May 11 07:58:09 2023

@author: BigD
"""
#Import all moduels as required
import astropy as ap
from astropy.stats import poisson_conf_interval
from astropy.table import Table
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
from statistics import mean
from statistics import stdev
from scipy.stats import (moyal, invgamma, lognorm, burr, beta, gamma, expon, exponpow, chi2, cauchy) 
from scipy import stats
from fitter import Fitter, get_common_distributions, get_distributions


# STEP 1 ORGANISE THE DATA

# Import the crossmatched data 
data = Table.read('/Users/BigD/cloudstor/Data/EMU - GAMA - G23')
data = data.to_pandas()

#G23_data = Table.read('C:/Users/BigD/cloudstor/Data/GAMA/GAMA - Michael Brown/GAMA_G23_v01_mu3_20191202.fits')
#G23_data = G23_data.to_pandas()

#import the EMU data
EMU_data = Table.read('/Users/BigD/cloudstor/Data/EMU/G23-ASKAP-EMUES-master-cat.fits', format='fits')
EMU_data= EMU_data.to_pandas()

#merge these two groups so that you have a ~40,000 sources table with 93 columns.
data = pd.merge(data, EMU_data, how="outer", on = 'Source_Name')
data = data[['Source_Name','RA_y','E_RA_y','DEC_y','E_DEC_y' ,'Total_flux_y','E_Total_flux_y','Z','NQ','Separation_x']]
data = data.rename(columns = {"RA_y":"RA", 'E_RA_y':'E_RA', 'DEC_y':'DEC', 'E_DEC_y':'E_DEC','Total_flux_y':'Total_flux','E_Total_flux_y':'E_Total_flux','Separation_x':'Separation' })

#del(EMU_data, data)

#We now have the crossmatched data set of EMU and G23, outlining:
print(list(data))

#create a column containing the rel error
data['rel_err']=data['E_Total_flux']/data['Total_flux']


#%% STEP 2 Visualise the data

from observationsofdataset import Visualiseradiodata

Visualiseradiodata(data)

#we can see that there needs to be a lot of completness corrections

#%% STEP 3 CREATE THE PDFS FOR MISSING REDSHIFT AND REDSHIFT ERRORS
#NOTE THIS STEP WILL CREATE A DATAFRAME DF THAT IS POSITIVE FLUX AND Z>0

#3.1 create pdfs for the redshift
from completenesscorrection import getpdfs
tol = 10 #the minimum number of data points in each bin to form a probability distribution function
n = 10 #the number of magnitude bins the data will be seperated into
df, pdfs, labels, use_labels, bins = getpdfs(data,n,tol)


#3.2 Show what portion of the data set was used to create pdf's
fig1, ax=plt.subplots(1,1)
for label in labels:
    #print(len(df[df['bin']==label]['mag']))
    ax.hist(df[df['bin']==label]['mag'], alpha = 1, rwidth=1, range= (min(df['mag']),max(df['mag'])), bins=200,label=label)
ax.set_xlabel('$Mag_{AB}$')
ax.set_ylabel('Counts')
plt.suptitle('The different magnitudes that created the pdfs')
plt.legend(fontsize = 10)


v = np.sum([df['bin'].str.contains(label).sum() for label in use_labels])#the number of data points each mag bin collected
v = "{0:.3%}".format(v/len(df))
print(f"pdf's were calculated from {v} of the datapoints that have a positive flux and greater than zero redshift")


#%% STEP 4 FILL IN THE MISSING DATA USING THE CREATED PDF'S

#Calculate the magnitudes for each data point
#Convert everything to AB magnitude
mag = np.add(25, np.multiply(-2.5, np.log10(data['Total_flux'])))
#add the column of magnitude to the dataset
data['mag'] = list(mag)

#Use the bins that the pdf's were created on, however add an upper and lower bound [ 0,50]
# so to capture anything that falls outside the range covered in the spectro sample
bins.append(50) #upper boundary
bins.insert(0,0) #lower boundary
#We must also create an upper bound for our labels
labels.append(str(labels[-1].replace('<',">")))


#Bin in accordance with the labels the pdf's were calculated for
data['bin'] = pd.cut(np.array(data['mag']), bins= bins, labels = labels[0:])
#Check the distribution
print(data['bin'].value_counts(dropna=False))  #Check the distribution of the bins 

#Count what percentage of data the pdf's don't account for
v = sum([data['bin'].value_counts(dropna=False)[label] for label in use_labels])
v = "{0:.3%}".format(v/len(data))
print(f"The pdf's can be utilised for {v} of the EMU data")


# 4.1 Fill in the missing data and draw comparison of the mock and given data
from fillinmissing import Fillinmissing
df= Fillinmissing(data, bins, labels, use_labels, pdfs)


#THIS IS WHERE I'M LOSING MY BIG FLUX READINGS< LOSING ABOUT 590 readings with only a 98.475

#For now this is good enough lets continue on, with the 98.475% data for the luminosity functions.
df = df[df['Z']>0]

#Display a magnitude vs redshift graph for the filled in data
fig2, ax =plt.subplots(1,1)
ax.scatter(df['Z'],np.log10(df['Total_flux']))
ax.set_xlabel('Redshift')
ax.set_ylabel('Flux (Jy)')


#%% For now this is good enough lets continue on, with the 98.475% data for the luminosity functions. #99.892%
#reset the index of the dataframe
df=df.reset_index(drop=True)


#%% STEP 5 Calculate the luminosity's for everything

#5.1 Find the turnaround of the flux

fig1, ax1 = plt.subplots()
counts, bin_edges = np.histogram(np.log10(df['Total_flux']), bins=200)
ax1.hist(np.log10(df['Total_flux']), bins = 200)
ax1.set_title('The total flux as measured by EMU')
ax1.set_xlabel('$Log_{10} S (Jansky)$')
ax1.set_ylabel('counts') 

idx = np.where(counts==max(counts))[0][0]
flux_limit=(10**bin_edges[idx])
print('the flux limit is ', ("{:.5f}".format(10**bin_edges[idx])))

#5.2 To find the flux limit lets reduce the data set to those values with only a positive redshift
df=df[df['Z']>0]
#reset the index
df=df.reset_index(drop=True)

fig2, ax1 = plt.subplots()
ax1.scatter(df['Z'],np.log10(df['Total_flux']))
ax1.set_title('The distribution of flux readings across\n the EMU GAMA Survey')
ax1.set_xlabel('Redshift')
ax1.set_ylabel('$Log_{10} S (Janksy)$')

#5.4 Let's calculate the luminosity for everything
#According to Gürkan the flux limits is 0.2mJy or 200uJy
flux_limit = 200e-6

#set parameters
radio_index=-0.7
#What is the survey radio frequency
EMU_Freq = 0.8875#Ghz #997 Mhz
#What is the frequency we wish to compare with (literature is normally 1.4Ghz)
Comp_Freq = 1.4#Ghz


from astropy.cosmology import WMAP9 as cosmo
ld=[]
for x in range(len(df)):
    ld.append(cosmo.luminosity_distance(df['Z'][x]).value * 3.086e22) #convert from MPc to m.
df['ld'] = ld

#Calculate the luminosity at 1.4Ghz
df['lum'] = 4*np.pi*(df['ld']**2)/((1+df['Z'])**(1+radio_index)*(EMU_Freq/Comp_Freq)**radio_index)*(df['Total_flux']*1e-26)
Min_lum_line = 4*np.pi*(df['ld']**2)/((1+df['Z'])**(1+radio_index)*(EMU_Freq/Comp_Freq)**radio_index)*(flux_limit*1e-26)

#2.4 Plot the results
fig3, [ax1,ax2] = plt.subplots(1,2)
fig3.suptitle('The distribution of readings across\n the EMU GAMA Survey', fontsize=15)
ax1.scatter(df['Z'],np.log10(df['lum']))
ax1.set_xlabel('Redshift')
ax1.set_ylabel('$Log_{10} L (Watts)$')
ax1.scatter(df['Z'], np.log10(Min_lum_line), s=2, label = str(flux_limit*1000) + "mJy")
ax1.legend(fontsize=10)
ax2.scatter(df['Z'], np.log10(df['Total_flux']), c = 'r')
ax2.set_xlabel('Redshift')
ax2.set_ylabel('$Log_{10} S (Janksy)$')


#%% STEP 6 Calculate Vmax using Novak Vmax method

#6.1 Define our Redshift bins
redshift_bins=[0.0,0.1,0.4,0.6,0.8,1.0,1.3,1.6,2.0]

#7.2 Create our labels to slice up the data
z_labels=['<' + str(redshift_bins[0])] #an array to create labels of the form <0.4, <0.6 etc, that will bin
          # each data point starting at 0.0
for ii in range(len(redshift_bins)-1):
    z_labels.append('<' + str(redshift_bins[ii+1]))
#Give each data point an according bin
df['z_bin'] = pd.cut(np.array(df['Z']), bins=redshift_bins, labels=z_labels[1:]) #note, we need to remove the 0.0 from the labels for
df['min_z'] = pd.cut(np.array(df['Z']), bins=redshift_bins, labels = redshift_bins[:-1])    


print(df['z_bin'].value_counts(dropna=False))  #Check the distribution of the bins 

Vmax_method="Observable"

if Vmax_method == "Novak":
    
    
    #6.1.1. Parameters for that dictate the correction
    #2.6 degrees squared for the Cosmos field 
    survey_area = 82.7
    survey_area = survey_area/41253 #portion of the sky surveyed.
    
    #Note the correction factor is not needeed
    # correction = survey_area * df['radio_cc'] * df['op_cc'] #eq 5. Novak 2017.
    # df['corr'] = correction
    
    #6.1.2 find which bin each redshift value is in
    z = df['Z']
    bin_num = []
    
    for value in z:
        for ii in range(len(redshift_bins)):
            if value<redshift_bins[ii]:
                bin_num.append(ii-1)
                break
        else:
            bin_num.append(-1)        
    #every point now has an index for which bin it is in, if bin number -1, then galaxy is outside of redshift bins
    
    #6.2 Calculate the space contained within each redshift bin using the method outlined in Novak 2017. (Sliced integral approx)
    Vbin=[]
    stepsize=0.01
    for n in range(len(redshift_bins)-1):
        z_range = np.arange(redshift_bins[n],redshift_bins[n+1]+stepsize,stepsize)
        #z_range = redshift_bins[n],redshift_bins[n+1]
        Volume=[]
        for i in range(len(z_range)-1): #Run through z_range, and sum up the Volume eq 8 Enia et. al '22
            Volume1= (4*np.pi/3) * cosmo.comoving_distance(z_range[i])**3
            Volume2= (4*np.pi/3) * cosmo.comoving_distance(z_range[i+1])**3
            Volume.append((Volume2 - Volume1).value)
        Vbin.append(sum(Volume))
        
    #Lets add a fail safe for things outside the redshift bins
    Vbin.append(0)
        
    # 6.3 Knowing which bin each galaxy is in, and each Volume contained in each bin vMax can be calculated.
    Vmax=[]
    
    for i in range(len(df)):
        Vol = Vbin[bin_num[i]]
        #Vmax.append(Vol* df['corr'][i])
        Vmax.append(Vol*survey_area)
    
    # We calculate a Vmax value for each and can add it to the database
    df['Vmax']=Vmax

if Vmax_method == "Observable":
    
    
    dmax = []
    dmax = np.sqrt(df['lum']*((1+df['Z'])**(1+radio_index)) /((flux_limit*1e-26) * (4*np.pi ))) #This will be in metres
    dmax=dmax * 3.24078e-23 #convert to Mpc
    df['dmax']= dmax

    # Calculate the seven z min distance each box is split up into
    min_d = []
    for value in redshift_bins[:-1]:
        min_d.append(cosmo.comoving_distance(value).value)
    
    #Create an array to store the minimum and actual distances in Mpc using the minimum redshifts of the redshift bins and redshit values.
    dmin = []
    dactual=[]
    for x in range(len(df)):
        dmin.append(cosmo.comoving_distance(df['min_z'][x]).value) #Using Cosmo luminosity distance gives us the answer in Mpc.
        dactual.append(cosmo.comoving_distance(df['Z'][x]).value)
    df['dmin']=dmin
    df['dactual']=dactual

    #This is the distance we can use to calculate the Vmax for each object, using the area of sky surveyed.
    #survey_area = (11**2)/(60**2) #convert from  sqaure of 11x11 arc minutes to square degree

    #Tanias paper 82.7 EMU Data covering
    #This should be 50.6 degrees squared for the G23 field 
    survey_area = 82.7
    survey_area = survey_area/41253 #portion of the sky surveyed.
    
    # We can now calculate the Maximum Volume each galaxy could've been observed in.
    Vmax = (4/3) * np.pi * (df['dmax'])**3 * survey_area
    Vmin = (4/3) * np.pi * (df['dmin'])**3 * survey_area

    # We calculate a Vmax value for each and can add it to the database
    df['Vmax']=Vmax
    df['Vmin']=Vmin
    
    #Subtract the Vmin from the Vmax so that the furtherest distance observable from the given bin
    #can be calculated 
    df['Vmax']=df['Vmax']-df['Vmin']
    
    
#%% Step 7 Classify the data


#7.2 Create our labels to slice up the data
z_labels=['<' + str(redshift_bins[0])] #an array to create labels of the form <0.4, <0.6 etc, that will bin
          # each data point starting at 0.0
for ii in range(len(redshift_bins)-1):
    z_labels.append('<' + str(redshift_bins[ii+1]))
#Give each data point an according bin
df['z_bin'] = pd.cut(np.array(df['Z']), bins=redshift_bins, labels=z_labels[1:]) #note, we need to remove the 0.0 from the labels for
df['min_z'] = pd.cut(np.array(df['Z']), bins=redshift_bins, labels = redshift_bins[:-1])    


print(df['z_bin'].value_counts(dropna=False))  #Check the distribution of the bins 


#%% STEP 8 Calculate Phi
#How many bins do you want for our lum funcs?
n_bins=10
#Do we want error bars
error_bars='y'
#Do we want Novaks published data to compare
lit="Novak"
#Check the data
print(df['z_bin'].value_counts(dropna=False))  #Check the distribution of the bins 


if lit == "Enia":
    redshift_bins=[0.0,0.4,0.7,1.0,2.0,3.0,10.0] #From Enia 2022
elif lit =="Novak":
    redshift_bins=[0.0,0.1,0.4,0.6,0.8,1.0,1.3,1.6,2.0] #Novak amended cut off with 2.0->10
    
#7.2 Create our labels to slice up the data
z_labels=['<' + str(redshift_bins[0])] #an array to create labels of the form <0.4, <0.6 etc, that will bin
              # each data point starting at 0.0
for ii in range(len(redshift_bins)-1):
    z_labels.append('<' + str(redshift_bins[ii+1]))
#Give each data point an according bin
df['z_bin'] = pd.cut(np.array(df['Z']), bins=redshift_bins, labels=z_labels[1:]) #note, we need to remove the 0.0 from the labels for
df['min_z'] = pd.cut(np.array(df['Z']), bins=redshift_bins, labels = redshift_bins[:-1])    

print(df['z_bin'].value_counts(dropna=False))  #Check the distribution of the bins 


#4.1 Create an array to store the Phi and Lum values that are to be plotted
Phi_arr = []
Lum_arr = []
error_arr=[] 
counts_arr=[] #An array to store the counts for each readshift bings
#Since we wish to do a log plot we will need to define the upper and lower limits seperately for an assymetircal graph
upper_err_arr=[] #Array to store all the upper errors arrays for each redshift bin
lower_err_arr=[] #Array to store all the lower errors arrays for each redshift bin

#4.2 Loop through each redshift bin and calculate phi, the lumminosity data and error for phi for each redshift bin
for x in range(len(z_labels[1:])):
    label = z_labels[x+1]
    zslice = df[df['z_bin']==label] #select our first redsfhit bin i.e. 0<z<label[1]
    zslice =zslice[['lum','Total_flux','Vmax','Z']] #take the useful values, i.e. luminosity and Vmax values
    #4.2.1 Not that we have removed all the outliers we can calculate the luminosity
    lum = zslice['lum']#select the luminosity data for the redshift bin
    V_list = zslice['Vmax'] #select the redshift data for the redshift bin
    print('For redshift between ' + str(z_labels[x]) + ' and ' +str(label))
    
    #4.2.1. Find any outliers in the luminosity data using basic Standard Deviation, replace later
    # with Grubbs test for outliers
    mean = np.mean(lum)
    std = np.std(lum)
    #Find any outliers outside 3 standard deviations
    outlier=[]
    for i in lum:
        if i > mean + 3*std or i < mean - 3*std:
            outlier.append(i)
    #remove outliers from the luminosity list
    for value in outlier:
        zslice.drop(zslice[zslice['lum'] == value].index, inplace = True)
             
    print("There were " + str(len(outlier)) +" outliers removed from data list " +
          str(len(lum)) +" long")
    
    
    #4.2.1 Not that we have removed all the outliers Redefine the luminosity and V_list
    lum = zslice['lum']#select the luminosity data for the redshift bin
    V_list = zslice['Vmax'] #select the redshift data for the redshift bin

    #4.2.2 Bin all the luminosities in the chosen redshift bin   
    counts, bin_edges = np.histogram(np.log10(lum), bins=n_bins)
    print('The number of galaxies in each bin is ' + str(counts)) #check to see how many galaxies fall in each luminosity bin for this slice
     
    #Lets order the zslice unsing the bin edges and counts
    zslice = zslice.sort_values(by=['lum'])
    # Calculate phi and the err for each bin
    phi =[] #an array to store the phi values for the chosen luminosity bins
    err=[] #an array to store the error values for the chosen luminosity bins
    l_half=[] #an array to store the width of each luminosity bin
    l_average=[]
    zero_count=0 # a variable to account for any bins that don't have a luminosity reading

    #4.2.3 Loop through each bin of the histogram and calculate phi and luminosity values
    for n in range(n_bins):

        if counts[n] > 0:
            delta_l = bin_edges[n+1]-bin_edges[n] #remember to remove the log
            #Create the luminosity reading
            l_half.append((bin_edges[n+1]+bin_edges[n])/2)
            l_average.append(np.log10(np.mean(zslice['lum'][0:counts[n]])))
            #Create our phi value
            phi.append(1/delta_l * sum((1/(zslice['Vmax']))[0:counts[n]]))
            #Calculate the error bars
            error=(1/delta_l * np.sqrt(sum((1/(zslice['Vmax']**2))[0:counts[n]])))
            if counts[n] <=10: #check if small number poisson statistics
                err.append(phi[n-zero_count]-phi[n-zero_count]*(ap.stats.poisson_conf_interval(counts[n]))[0]/counts[n]) #add the small number error to the err array
            else:
                err.append(error) #Novak 2017 eq (6)
            #now to chop the number of points used from the zslice for the sake of the loop
            zslice=zslice[counts[n]:]
        else:
            zero_count+=1
    
    counts_arr.append(counts)
    Phi_arr.append(phi)
    error_arr.append(err)
    Lum_arr.append(l_average) 
    

upper_err_arr=np.add(Phi_arr,error_arr)
lower_err_arr=np.subtract(Phi_arr,error_arr)


#NOTE SOME ERRORS MAY BE LARGER THAN THEIR PHI VALUES. i.e. if PHI is 1.6e-3, but the error is 
#1.8e-3, that simply is a issue with the PHI valye being so close to zero. 
#I must create a clause s.t. if error is larger the phi value, the error must equal the phi value.
#OR if the lower err_arr is <0, set it to 0, or set the error to be exactly equal to the value.
for jj in range(len(z_labels[1:])):
    lower_err_arr[jj][lower_err_arr[jj]<0]=0

#%%
#4.3 Graph the values of Phi and Lum for all redshift bins
counter = len(z_labels)-1 #Find how many redshift bins we wish to graph

#4.3.1 Plot
fig, ax = plt.subplots(2,int(counter/2), sharex=(True), sharey=True)
fig.suptitle('Luminosity functions for various redshift \n slices using the EMU-G23 data', fontsize=10)
fig.text(0.5, 0.02, r'$logL_{1.4GHz}[W Hz^{-1}]$', ha='center', fontsize =10)
fig.text(0.02, 0.5, r'$log\Phi[Mpc^{-3}dex^{-1}]$', va='center', rotation='vertical', fontsize=10)
plt.subplots_adjust(bottom=0.15, left=0.11, top=0.9, wspace=0, hspace=0)
plt.yscale('log')
   
for ii in range(counter): #RUN THE LOOP THROUGH THE LABELLED/FILTERED INSTEAD OF SLICES
    textstr=str(z_labels[ii][1:]) + '<z' +str(z_labels[ii+1])
    print(ii)
    
    #Create out asymetricerror
    asymetricerror=[lower_err_arr[ii],upper_err_arr[ii]]
    
    print(asymetricerror)
    
    # Plot the resulting phi and Luminosity on the two rows of the figure
    if ii < int(counter/2):
        ax[0,ii].scatter(Lum_arr[ii],Phi_arr[ii], label='EMU G23')
        ax[0,ii].tick_params(axis='both', which='major', labelsize=10)
        ax[0,ii].tick_params(axis='both', which='minor', labelsize=10)
        ax[0,ii].label_outer()
        ax[0,ii].grid()
        ax[0,ii].text(0.02, 0.1, textstr, transform=ax[0,ii].transAxes, fontsize=10, verticalalignment='bottom', color='red',
                      bbox = dict(facecolor='white', edgecolor='red', pad=2.0))
        if error_bars == "y":
            ax[0,ii].errorbar(Lum_arr[ii],Phi_arr[ii], yerr=np.array(asymetricerror), ls='none')
        
        
    else:
        #ax[1,int(ii-counter/2)].set_yscale('log')
        ax[1,int(ii-counter/2)].scatter(Lum_arr[ii],Phi_arr[ii])
        ax[1,int(ii-counter/2)].label_outer()
        ax[1,int(ii-counter/2)].text(0.02, 0.9, textstr, transform=ax[1,int(ii-counter/2)].transAxes, fontsize=10, 
                    verticalalignment='bottom', color='red', bbox = dict(facecolor='white', edgecolor='red', pad=2.0))
        ax[1,int(ii-counter/2)].grid()
        ax[1,int(ii-counter/2)].tick_params(axis='both', which='major', labelsize=10)
        ax[1,int(ii-counter/2)].tick_params(axis='both', which='minor', labelsize=10)
        if error_bars == "y":
            ax[1,int(ii-counter/2)].errorbar(Lum_arr[ii],Phi_arr[ii], yerr=np.array(asymetricerror), fmt='o', ls='none')
        
        
    
    
    print('The Phi values for each bins are ', ['%.2f' % np.log10(elem) for elem in Phi_arr[ii]])
    print("The number of points in each bin are ", counts_arr[ii])
    print('The data is plotted at luminosities of ', ['%.2f' % elem for elem in Lum_arr[ii]])
    print('\n')
    
#Novak Findings Overlay
if lit=="Novak":
        Novak_L = [[21.77,22.15,22.46,22.77,23.09,23.34],[22.29,22.54,22.80,23.04,23.31,23.68],
                   [22.61,22.84,23.11,23.40,23.71,24.06],[22.85,23.05,23.30,23.54,23.81,24.11],
                   [23.10,23.31,23.57,23.84,24.06,24.38],[23.32,23.53,23.81,24.15,24.39,24.82],
                   [23.55,23.72,23.98,24.28,24.536,24.90]] #Need to change these to Log Luminosities
       
        Novak_Phi = [[-2.85,-2.88,-3.12,-3.55,-4.05,-4.63],[-2.97,-3.19,-3.33,-3.67,-4.32,-5.05],
                     [-2.89,-3.13,-3.47,-3.99,-4.68,-5.43],[-3.01,-3.13,-3.45,-3.85,-4.31,-4.89],
                     [-3.19,-3.42,-3.86,-4.15,-4.74,-5.25],[-3.36,-3.55,-4.10,-4.53,-5.30,-5.94],
                     [-3.47,-3.66,-4.15,-4.56,-5.06,-5.86]] 
        Novak_Phi = [np.power(10, x) for x in Novak_Phi]
        
        ax[0,1].scatter(Novak_L[0],Novak_Phi[0], label = "Novak 2017")
        ax[0,2].scatter(Novak_L[1],Novak_Phi[1])
        ax[0,3].scatter(Novak_L[2],Novak_Phi[2])
        ax[1,0].scatter(Novak_L[3],Novak_Phi[3])
        ax[1,1].scatter(Novak_L[4],Novak_Phi[4])
        ax[1,2].scatter(Novak_L[5],Novak_Phi[5])
        ax[1,3].scatter(Novak_L[6],Novak_Phi[6])
        
#Enia Findings Overlay
if lit =="Enia":  
    Enia_L=[[21.48,21.99,22.42,22.84],[22.16,22.55,22.96,23.38],
            [22.58,22.88,23.26,23.63],[23.04,23.57,23.96,24.35],
            [23.59,23.94,24.27,24.60],[23.70,24.09,24.27,24.69]]
    Enia_Phi=[[-2.40,-2.77,-3.20,-3.42],[-2.38,-2.75,-3.19,-3.80],
              [-2.54,-2.84,-3.19,-3.99],[-2.48,-3.55,-4.16,-4.72],
              [-2.94,-3.72,-4.26,-4.81],[-3.26,-3.98,-4.52,-4.98]]
                    
                        
    ax[0,0].scatter(Enia_L[0],Enia_Phi[0])
    ax[0,1].scatter(Enia_L[1],Enia_Phi[1], label = "Enia 2022")
    ax[0,2].scatter(Enia_L[2],Enia_Phi[2])
    ax[1,0].scatter(Enia_L[3],Enia_Phi[3])
    ax[1,1].scatter(Enia_L[4],Enia_Phi[4])
    ax[1,2].scatter(Enia_L[5],Enia_Phi[5])
      
        

handles, labels = ax[0,1].get_legend_handles_labels()
fig.legend(handles, labels,bbox_to_anchor = (1,1.03), loc='upper right', fontsize=9)


 #%%LETS TAKE THE FIRST 4 BOXES THAT ARE VALID TO USE

Phi=Phi_arr[0:4]
Lum = Lum_arr[0:4]
Lower_err = lower_err_arr[0:4]
Upper_err = upper_err_arr[0:4]



#%%Lets look at these 4 areas

# Create subplots with shared y-axis
fig, axs = plt.subplots(2, 2, figsize=(10, 8), sharey=True, sharex=True)
fig.suptitle('A closer look at the luminosity functions of EMU and Novak')

# Flatten the axs array for easier indexing
axs = axs.flatten()

# Iterate over the datasets and create plots
for i, ax in enumerate(axs):
    x = Lum[i]
    y = Phi[i]
    lower_err = Lower_err[i]
    upper_err = Upper_err[i]  
    # Plot the data points with asymmetric error bars
    ax.errorbar(x, y, yerr=[lower_err,upper_err], fmt='o', capsize=3, label='EMU')
    #ax.errorbar(x, y, yerr=[0.3*data for data in y], fmt='o', capsize=3) #custom error.   
    # Set plot labels and title
    ax.set_title(str(redshift_bins[i])+'<z<'+str(redshift_bins[i+1]))
    ax.set_yscale('log')  
    # Add gridlines
    ax.grid(True)
 # Add secondary data to the final three plots
    if i > 0:
        ax.scatter(Novak_L[i-1],Novak_Phi[i-1], c='orange', label='Novak')
        ax.legend()

# Set a common y-axis label
fig.text(0.04, 0.5, 'Phi', va='center', rotation='vertical')
fig.text(0.5, 0.04, 'Lum', va='center', rotation='horizontal')
# Adjust spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()

#%% GO TO 4_PARAMETER FIT IN ULTRANEST






#%% Wish to fit a Saunders function, from Saunders et al 1990, Novak et al. 2017

